In [1]:
import sys
import xlrd
import pandas as pd
import numpy as np
from numpy import median
import scipy
import os
import fnmatch 
import random
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import statistics
from scipy import signal
import math
from random import sample 

In [2]:
# Range of genes: length in bp
d1 = 1500
d2 = 3500
num_c = 2
m1 = 500
m2 = 1500
viewer = 1500
# Ribosome counts directory
csv_dir = '/Users/jiz225/Desktop/ZidLab/polarity/output/Anna/'
# Delta Dhh1 file
dhh1file = '/Users/jiz225/Desktop/ZidLab/polarity/Dhh1_Length_Short.xlsx'
# Mitochondia genes
mitofile = '/Users/jiz225/Desktop/ZidLab/polarity/Williams_mito_Proxi_ribosom_Profiling.xls'

In [3]:
red = ['YLL026W', 'YDR258C', 'YPL240C', 'YBR169C', 'YER103W', 
       'YGR142W', 'YDR171W', 'YCR021C', 'YBR072W', 'YFL014W', 'YLR327C']
isred = []
df_dhh1_all = pd.read_excel(dhh1file, index_col=None)
for i in df_dhh1_all.get('Name'):
    if i in red:
        isred.append('y')
    else:
        isred.append('n')
df_dhh1_all['Type'] = isred
print(max(df_dhh1_all.iloc[:,3]), min(df_dhh1_all.iloc[:,3]))
df_dhh1 = df_dhh1_all[(df_dhh1_all.iloc[:,3] > d1) & (df_dhh1_all.iloc[:,3] < d2)]
print(df_dhh1.columns)

df_dhh1_ex = df_dhh1[(df_dhh1.iloc[:,-1] == 'n')]
print('number of genes excluding red gens',len(df_dhh1_ex))
df_dhh1_in = df_dhh1
print('number of genes including red gens',len(df_dhh1_in))

dhh1_top = df_dhh1.sort_values(by='Delta_Dhh1', ascending=False).iloc[:600,:]
print(dhh1_top.iloc[:,6].quantile([0.05,0.25,0.5,0.75,0.995,1]))
dhh1_bot = df_dhh1.sort_values(by='Delta_Dhh1', ascending=True).iloc[:800,:]
print(dhh1_bot.iloc[:,6].quantile([0.05,0.25,0.5,0.75,0.995,1]))

14733 201
Index(['Name', 'Code', 'Annotation', 'ORF Length', 'YEP(ip-total) -G Dhh1',
       'YPD(ip-total) +G Dhh1', 'Delta_Dhh1', 'LengthPredicted Dhh1',
       'D_predicted', 'tai', 'BY_Ribo_B_Polarity', 'BY_Ribo_A_Polarity',
       'Ribosome Occupancy +G Ribo/mRNA - 1P/1Tt', 'Ribo +G - 1Pd_C',
       'mRNA +G - 1Ttd-Ribozero', '-G/+G mRNA - 4Tdat/1Tdat',
       '-G/+G Ribosome occupancy', 'Type'],
      dtype='object')
number of genes excluding red gens 1602
number of genes including red gens 1607
0.050    0.643862
0.250    0.766110
0.500    0.941318
0.750    1.197433
0.995    2.505705
1.000    3.178599
Name: Delta_Dhh1, dtype: float64
0.050   -0.866660
0.250   -0.210997
0.500    0.060266
0.750    0.264509
0.995    0.440633
1.000    0.444185
Name: Delta_Dhh1, dtype: float64


In [10]:
df_mito = pd.read_excel(mitofile,index_col=None)
df_mito = df_mito.rename(columns={'#gene':'Name'})
df_mito = df_mito[(df_mito.Name.isin(df_dhh1.Name)) & (df_mito.mitop2 == 'yes')]  
df_nonmito = df_dhh1[(~df_dhh1.Name.isin(df_mito.Name))]
print('df_mito', df_mito.shape, df_mito.columns)
print('df_nonmito', df_nonmito.shape, df_nonmito.columns)
df_mito_diff = df_mito[(df_mito['minus.chx.enriched'] == 'no') & (df_mito['plus.chx.enriched'] == 'yes')]
print('df_mito_diff ', df_mito_diff.shape, max(df_mito_diff['aa.length'])*3,min(df_mito_diff['aa.length'])*3)
#df_mito_yes = df_mito[(df_mito['minus.chx.enriched'] == 'yes') & (df_mito['plus.chx.enriched'] == 'yes')]
df_mito_yes = df_mito[(df_mito['minus.chx.enriched'] == 'yes')]
print('df_mito_yes ', df_mito_yes.shape, max(df_mito_yes['aa.length'])*3,min(df_mito_yes['aa.length'])*3)
#df_mito_no = df_mito[(df_mito['minus.chx.enriched'] == 'no') & (df_mito['plus.chx.enriched'] == 'no')]
df_mito_no = df_mito[(df_mito['plus.chx.enriched'] == 'yes')]
print('df_mito_no ', df_mito_no.shape, max(df_mito_no['aa.length'])*3,min(df_mito_no['aa.length'])*3)

df_mito (136, 29) Index(['Name', 'common.name', 'aa.length', 'mitop2', 'mito.compartment',
       'mito.GO', 'gfp.localization', 'unannonated.enriched', 'mitoprot.score',
       'signalp.predicted', 'glycosylated', 'gene.ohnolog',
       'minus.chx.enriched', 'plus.chx.enriched', 'om45.2mchx.total.cts',
       'om45.2mchx.total.rpkm', 'om45.2mchx.pulldown.cts',
       'om45.2mchx.pulldown.rpkm', 'om45.2mchx.log2enrich',
       'om45.2m.total.cts', 'om45.2m.total.rpkm', 'om45.2m.pulldown.cts',
       'om45.2m.pulldown.rpkm', 'om45.2m.log2enrich', 'ssh1.2mchx.total.cts',
       'ssh1.2mchx.total.rpkm', 'ssh1.2mchx.pulldown.cts',
       'ssh1.2mchx.pulldown.rpkm', 'ssh1.2mchx.log2enrich'],
      dtype='object')
df_nonmito (1471, 18) Index(['Name', 'Code', 'Annotation', 'ORF Length', 'YEP(ip-total) -G Dhh1',
       'YPD(ip-total) +G Dhh1', 'Delta_Dhh1', 'LengthPredicted Dhh1',
       'D_predicted', 'tai', 'BY_Ribo_B_Polarity', 'BY_Ribo_A_Polarity',
       'Ribosome Occupancy +G Ribo/mRNA -

In [11]:
os.chdir(csv_dir)
wd = os.getcwd()

csv_list=[]
tag = 'longmers'
for root,dirs,files in os.walk(wd):
    for file in files:
        if file.split('.')[-1] == "csv" and tag in file:
            csv_list.append(csv_dir + file)

csv_list_s = sorted(csv_list)
print(len(csv_list_s))

5


In [12]:
keys = []
values_list = []
fit_list = []
times = []
llists = []
med = []
colors = ['blue','red','green','purple']
#df_dhh1 = dhh1_top
#df_mito = df_nonmito
for i in range(1,len(csv_list_s)):
    file = csv_list_s[i]
    time = file.split('NI-')[1].split('_RPF')[0]
    times.append(time)
    df_all = pd.read_csv(file)
    df = pd.merge(df_all, df_mito['Name'], how='inner', on=['Name'])
    df_mito = df_mito[df_mito.Name.isin(df.Name)] 
    #non_mito_sample = sample(np.unique(np.array(df_mito['Name'])).tolist(),75)
    #df = df[df.Name.isin(non_mito_sample)]  
    #print(non_mito_sample)
    '''
    df = pd.merge(df_all, df_dhh1_in[['Name','Code']], how='inner', on=['Name'])
    df = df.drop(columns=['Name'])
    df = df.rename(columns={'Code':'Name'})
    cols = df.columns.tolist()
    col = cols[-1:] + cols[:-1]
    df = df[col]
    '''
    reads_dict = df.groupby('Name').agg({'Counts': np.sum, 'RelativePosition': pd.Series.nunique})
    reads_dict['ReadsDensity'] = reads_dict['Counts']/reads_dict['RelativePosition']
    spread_rp = reads_dict.iloc[:,-2].quantile([0.25, 1])
    #print(spread_rp)
    
    counts = []
    for k in range(len(df)):
        name = df.iloc[k,0]
        count = df.iloc[k,-1]
        counts.append(count/reads_dict.loc[name,'ReadsDensity'])
    df['NormCounts'] = counts  
    
    low = spread_rp.get(key=0.25)
    high = spread_rp.get(key=1)
    reads_dict = reads_dict[(reads_dict.RelativePosition > low) & (reads_dict.RelativePosition < high)]
    reads_dict_1 = df.groupby(['Name','RelativePosition']).agg({'NormCounts': np.sum}).reset_index()
    reads_dict_1 = reads_dict_1[reads_dict_1.Name.isin(reads_dict.index)]
    spread_nc = reads_dict_1.iloc[:,-1].quantile([0.05,0.1,0.15,0.25,0.5,0.65,0.75,0.85,0.95,1])
    #print(spread_nc)
    reads_dict_1 = reads_dict_1[reads_dict_1.NormCounts <= spread_nc.get(key=0.95)]   
    rbf_dict = reads_dict_1.groupby(['RelativePosition']).agg({"NormCounts": np.sum, "Name": pd.Series.nunique})
    #df = df[df.Name.isin(reads_dict.index)&df.Position.isin(reads_dict_1.Position)]
    #rbf_dict = df.groupby(['RelativePosition']).agg({"NormCounts": np.sum, "Name": pd.Series.nunique})
    all_counts = reads_dict_1.groupby('Name').agg({'NormCounts': np.sum})
    counts_distribution = all_counts.iloc[:,-1].quantile([0.05,0.25,0.5,0.75,0.95])
    print('In sample ', time, ' number of genes used ', len(reads_dict))
    print('In sample ', time, ' ribosome counts distribution is')
    print(counts_distribution)
    reads = 0
    
    agg_dict = dict()
    for c in range(viewer):
        if c in rbf_dict.index:
            reads += rbf_dict.loc[c,'NormCounts']/rbf_dict.loc[c,'Name']
            #if c in range(m1,m2):
                #med.append(rbf_dict.loc[c,'NormCounts']/rbf_dict.loc[c,'Name'])
        if c % (num_c*3) == 0 and reads > 0:
            agg_dict.update({int(c/3):reads/(num_c*3)})
            if c in range(m1,m2):
                med.append(reads/(num_c*3))
            reads = 0
    
            
    keys = np.array(list(agg_dict.keys()))
    values = np.array(list(agg_dict.values()))
    values_list.append(values) 
    #break

In sample  811  number of genes used  54
In sample  811  ribosome counts distribution is
0.05     24.722414
0.25     42.269231
0.50     83.500000
0.75    219.997171
0.95    490.224608
Name: NormCounts, dtype: float64
In sample  813  number of genes used  53
In sample  813  ribosome counts distribution is
0.05     48.789831
0.25     75.551402
0.50    145.355191
0.75    328.118196
0.95    571.658976
Name: NormCounts, dtype: float64
In sample  815  number of genes used  54
In sample  815  ribosome counts distribution is
0.05     30.000000
0.25     56.334248
0.50    107.984720
0.75    262.166662
0.95    496.695237
Name: NormCounts, dtype: float64
In sample  817  number of genes used  53
In sample  817  ribosome counts distribution is
0.05     34.472727
0.25     60.000000
0.50    115.000000
0.75    233.225806
0.95    481.612408
Name: NormCounts, dtype: float64


In [13]:
overall = statistics.median(med)

In [15]:
all_l = []
l = []
fig = go.Figure()
colors = ['blue', 'red', 'green', 'purple', 'orange']
def func(x, a, b):
        return a * np.log(x) + b
    
for t in range(len(times)):
    sample_color = colors[t]
    x = keys
    y = values_list[t] - statistics.median(values_list[t]) + overall
    fig.add_trace(go.Scatter(mode='markers',x = x[:len(y)],y = y,marker=dict(color=sample_color),name= times[t]+'data points'))
    #fig.add_trace(go.Scatter(mode='lines',x=x,y=y,line=dict(color=sample_color),name= times[t]+'unsmoothed data'))
    popt, pcov = scipy.optimize.curve_fit(func,x[:len(y)],y)
    fit_y = func(x, *popt)
    fig.add_trace(go.Scatter(mode='lines',x = x[:len(fit_y)],y = fit_y,line=dict(color=sample_color),name= times[t]))
    #all_l.append(fit_y[:250])
             

fig.update_layout(
    title_text="Normalized read at codon position for midochondria genes between " + str(d1) + "bp and " + str(d2)+ "bp",
    xaxis_title="Codon position",
    yaxis_title="Normalized ribosome reads(unsmoothed)"
)   

fig.update_xaxes(range=[0, viewer/3])
#fig.update_yaxes(range=[0, ])

fig.show()

In [82]:
time_course = [0,1*60, 5*60, 15*60]
fit_y = 0.86
for i in range(1,len(time_course)):
    diff_idx = {}
    max_idx = 0
    for j in range(len(all_l[0])):
        if all_l[0][j] > max(all_l[0]):
            break
        if not not diff_idx:
            max_idx = max(diff_idx.keys())
        k = max([j+1,max_idx])  
        while k <= all_l[i].index(max(all_l[i])):
            if abs(all_l[i][k] - all_l[0][j]) <= 0.002 and k not in diff_idx.keys():
                diff_idx.update({k:(k-j)*num_c})
                #print(j, k)
                break
            k = k+1
    
    max_rate = sum(diff_idx.values())/len(diff_idx)/time_course[i]
    print(times[0], ' to ', times[i], 'avg max rate is ', max_rate)

for i in range(1,len(time_course)):
    diff_idx = {}
    max_idx = 0
    for j in range(len(all_l[i-1])):
        if all_l[0][j] > max(all_l[0]):
            break
        if not not diff_idx:
            max_idx = max(diff_idx.keys())
        k = max([j+1,max_idx])
        #print(j)
        while k <= all_l[i].index(max(all_l[i])):
            if abs(all_l[i][k] - all_l[i-1][j]) <= 0.002 and k not in diff_idx.keys():
                diff_idx.update({k:(k-j)*num_c})
                break
            k = k+1
    #print(diff_idx)
    min_rate = sum(diff_idx.values())/len(diff_idx)/(time_course[i] - time_course[i-1])
    print(times[i-1], ' to ', times[i], 'avg min rate is ', min_rate)

811  to  813 avg max rate is  1.0980276134122287
811  to  815 avg max rate is  0.13721943048576216
811  to  817 avg max rate is  0.036452790818988
811  to  813 avg min rate is  1.0980276134122287


ZeroDivisionError: division by zero

In [ ]:
overall_1 = np.quantile(med, [0.5])
overall_2 = np.quantile(med, [0.45])
overall_3 = np.quantile(med, [0.4])
overall_4 = np.quantile(med, [0.35])

In [ ]:
all_l = []
l = []
fig = go.Figure()
for t in range(len(times)):
    if t == 0:
        l = signal.savgol_filter(values_list[t] - statistics.median(values_list[t]) + overall_1,33,5)
        fig.add_trace(go.Scatter(x=keys, y=l,
                    mode='lines',
                    name= times[t]+' logphase'))
    elif t == 1:
        l = signal.savgol_filter(values_list[t] - statistics.median(values_list[t]) + overall_2,33,5)
        fig.add_trace(go.Scatter(x=keys, y=l,
                    mode='lines',
                    name= times[t]+' 1min -Glu'))
    elif t == 2:
        l = signal.savgol_filter(values_list[t] - statistics.median(values_list[t]) + overall_3,33,5)
        fig.add_trace(go.Scatter(x=keys, y=l,
                    mode='lines',
                    name= times[t]+' 5min -Glu'))
    else:
        l = signal.savgol_filter(values_list[t] - statistics.median(values_list[t]) + overall_4,33,5)
        fig.add_trace(go.Scatter(x=keys, y=l,
                    mode='lines',
                    name= times[t]+ ' 15min -Glu'))
    all_l.append(l.tolist()[:40])

fit_y = 0.9358
fig.add_shape(go.layout.Shape(type="line",x0=10,y0=fit_y,x1=490,y1=fit_y,
                              line=dict(color="Black",width=2.5,dash="dashdot")))
'''
fig.add_shape(go.layout.Shape(type="line",x0=10,y0=0.9438,x1=660,y1=0.9438,
                              line=dict(color="Black",width=2.5,dash="dashdot")))
fig.add_shape(go.layout.Shape(type="line",x0=180,y0=1.01,x1=180,y1=1.05,
                              line=dict(color="Blue",width=1.5,dash="dashdot")))
fig.add_shape(go.layout.Shape(type="line",x0=285,y0=1.01,x1=285,y1=1.05,
                              line=dict(color="Red",width=1.5,dash="dashdot")))
fig.add_shape(go.layout.Shape(type="line",x0=410,y0=1.01,x1=410,y1=1.05,
                              line=dict(color="Green",width=1.5,dash="dashdot")))
fig.add_shape(go.layout.Shape(type="line",x0=480,y0=1.01,x1=480,y1=1.05,
                              line=dict(color="Purple",width=1.5,dash="dashdot")))

fig.add_trace(go.Scatter(
    x=[220,330,415],
    y=[1.02,1.02,1.02],
    mode='text',
    name='Peak Elongation rate',
    text=['~1.75aa/s','0.52-0.76aa/s', '0.12-0.33aa/s'],
    textposition='top right',
    textfont=dict(size=13,color='#ff7f0e')
))
'''
fig.add_trace(go.Scatter(
    x=[60,130,220],
    y=[0.64,0.64,0.64],
    mode='text',
    name='Avg Elongation rate',
    text=['~1.037aa/s','0.15-0.31aa/s', '0.071-0.133aa/s'],
    textposition='top right',
    textfont=dict(size=13,color='#1f77b4')
))

#fig.update_xaxes(range=[10, 500])
#fig.update_yaxes(range=[0.6, 1.0])

fig.update_layout(
    title_text="Normalized read at codon position for all mitochondria genes between " + str(d1) + "bp and " + str(d2)+ "bp",
    xaxis_title="Codon position",
    yaxis_title="Normalized ribosome reads(smoothed)"
)       

fig.show()

In [48]:
time_course = [0,1*60, 5*60, 15*60]

for i in range(1,len(time_course)):
    diff_idx = {}
    max_idx = 0
    for j in range(len(all_l[0])):
        if all_l[0][j] > max(all_l[0]):
            break
        if not not diff_idx:
            max_idx = max(diff_idx.keys())
        k = max([j+1,max_idx])  
        while k <= all_l[i].index(max(all_l[i])) and all_l[i][k] < min(max(all_l[i]),fit_y):
            if abs(all_l[i][k] - all_l[0][j]) <= 0.005 and k not in diff_idx.keys():
                diff_idx.update({k:(k-j)*10})
                #print(j, k)
                break
            k = k+1
    
    max_rate = sum(diff_idx.values())/len(diff_idx)/time_course[i]
    print(times[0], ' to ', times[i], 'avg max rate is ', max_rate)

for i in range(1,len(time_course)):
    diff_idx = {}
    max_idx = 0
    for j in range(len(all_l[i-1])):
        if all_l[0][j] > max(all_l[0]):
            break
        if not not diff_idx:
            max_idx = max(diff_idx.keys())
        k = max([j+1,max_idx])
        #print(j)
        while k <= all_l[i].index(max(all_l[i])) and all_l[i][k] < min(max(all_l[i]),fit_y):
            if abs(all_l[i][k] - all_l[i-1][j]) <= 0.005 and k not in diff_idx.keys():
                diff_idx.update({k:(k-j)*10})
                break
            k = k+1
    #print(diff_idx)
    min_rate = sum(diff_idx.values())/len(diff_idx)/(time_course[i] - time_course[i-1])
    print(times[i-1], ' to ', times[i], 'avg min rate is ', min_rate)

NameError: name 'fit_y' is not defined